## Runs DE on different clusters. Also runs DE within each cluster for each condition

In [1]:
integrated_f = "/data2/mito_lineage/output/annotation/data/jan21_2021/MTblacklist/mergedSamples/allSamples.integrated.rds" 
outdir = "/data2/mito_lineage/output/annotation/data/jan21_2021/MTblacklist/mergedSamples/DE_minPct01"
sample_names = "Control,Flt3l"

# integrated_f = "/data/isshamie/mito_lineage/output/annotation/TcellDupi_may17_2021/MTblacklist/mergedSamples/allSamples.integrated.rds"
# outdir = "/data/isshamie/mito_lineage/output/annotation/TcellDupi_may17_2021/MTblacklist/mergedSamples/DE" 
# sample_names = "pre,post"


# Input info
#samples <- "J2,P2"
# sample_names = "Flt3l,Control"
# # Saving
# integrated_f = "/data/isshamie/mito_lineage/output/annotation/cd34norm/MTblacklist/mergedSamples/allSamples.integrated.rds" #"/data2/mito_lineage/Analysis/annotation/output/data/"
# outdir = "/data/isshamie/mito_lineage/output/annotation/cd34norm/MTblacklist/mergedSamples/DE"
# Parameters
nTop = 25000
assay="RNA"

minPct = 0.01
cores = 4


comps_f = ""
#cond.comparisons <- "A,preA,postA;B,preB,postB"

In [2]:
clust_outdir = file.path(outdir, "clusters")
clust_atac_outdir = file.path(outdir, "clusters_atac")
cond_outdir = file.path(outdir, "conditions_clusters")
cons_outdir = file.path(outdir, "conditions_conserved")
dir.create(outdir, showWarnings = FALSE)
dir.create(clust_outdir, showWarnings = FALSE)
dir.create(cond_outdir, showWarnings = FALSE)
dir.create(cons_outdir, showWarnings = FALSE)
dir.create(clust_atac_outdir, showWarnings = FALSE)

In [3]:
sample_names <- unlist(strsplit(sample_names, ",")[[1]])
#samples <- unlist(strsplit(samples, ","))

sample_names

[1] "Control" "Flt3l"

In [4]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")

library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The follow

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [5]:
integrated <- readRDS(integrated_f)#file.path(indir, paste0("allSamples.integrated.rds")))
integrated

An object of class Seurat 
169545 features across 18492 samples within 2 assays 
Active assay: RNA (20010 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

In [6]:
integrated[['peak_region_fragments']]

,peak_region_fragments
,<dbl>
Control_AAACGAAAGAGGTCCA-1,1330
Control_AAACGAAAGCGATACG-1,3462
Control_AAACGAAAGTCGTGAG-1,966
Control_AAACGAACAATAGTGA-1,2701
Control_AAACGAACACAATAAG-1,1138
Control_AAACGAACACTGATAC-1,878
Control_AAACGAACAGTAACCT-1,651
Control_AAACGAACATCGTGAT-1,7263
Control_AAACGAAGTAACGGTG-1,1042


In [7]:
integrated[[]]

,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,⋯,promoter_region_fragments,on_target_fragments,blacklist_region_fragments,peak_region_fragments,peak_region_cutsites,proj,ATAC_snn_res.0.8,seurat_clusters,nCount_RNA,nFeature_RNA
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>,<fct>,<dbl>,<int>
Control_AAACGAAAGAGGTCCA-1,Control,1357,1357,16929,2587,58,70,674,9629,3911,⋯,0,740,0,1330,2434,Control,5,5,1673,1406
Control_AAACGAAAGCGATACG-1,Control,3505,3505,45359,10546,184,184,2302,22247,9896,⋯,0,1611,0,3462,6167,Control,0,0,4315,3063
Control_AAACGAAAGTCGTGAG-1,Control,994,994,10177,2450,48,60,657,2135,4827,⋯,0,775,0,966,1768,Control,5,5,1731,1414
Control_AAACGAACAATAGTGA-1,Control,2687,2687,22452,4364,100,83,897,9233,7775,⋯,0,1239,0,2701,4951,Control,0,0,3113,2340
Control_AAACGAACACAATAAG-1,Control,1184,1184,11763,2003,43,57,680,5547,3433,⋯,0,621,0,1138,2019,Control,5,5,1559,1394
Control_AAACGAACACTGATAC-1,Control,954,954,9449,1971,59,21,489,3785,3124,⋯,0,517,0,878,1595,Control,2,2,1370,1207
Control_AAACGAACAGTAACCT-1,Control,693,693,6196,1097,19,17,288,2635,2140,⋯,0,397,0,651,1170,Control,8,8,1018,939
Control_AAACGAACATCGTGAT-1,Control,6800,6800,100024,36721,588,531,5317,36828,20039,⋯,0,3467,0,7263,13261,Control,12,12,8645,5001
Control_AAACGAAGTAACGGTG-1,Control,1074,1074,8454,1950,24,33,354,3729,2364,⋯,0,451,0,1042,1920,Control,7,7,1005,905


## Compute DE for each cluster

In [8]:
plotDE <- function(integrated, de.results, i, outdir){
    try
    {
    plot1 <- VlnPlot(
      object = integrated,
      features = rownames(de.results)[1],
      pt.size = 0.1,
      idents = i
    )
    plot2 <- FeaturePlot(
      object = integrated,
      features = rownames(de.results)[1],
      pt.size = 0.1
    )

    plot3 <- FeaturePlot(
      object = integrated,
      features = rownames(de.results)[2],
      pt.size = 0.1
    )
    
    plot1 | plot2 | plot3
    
    ggsave(file.path(outdir,paste0("cluster_",i,".DE.GeneActivity.top2.png")))
    
    return(c(plot1, plot2, plot3))
    }
    return
}

## Loop through each cluster and run DA for RNA-seq

In [9]:

cluster.ids <- sort(unique(integrated$seurat_clusters))
for (i in cluster.ids){ #or however many clusters you have
try({
    print(paste("cluster", i))
    da.peaks <- FindMarkers(
                  object = integrated,
                  ident.1 = i, #"CD4 Naive",
                  min.pct = minPct,
                  test.use = 'LR', latent.vars = 'peak_region_fragments'
                )
    print(file.path(clust_outdir, paste0("cluster_",i,".DE.GeneActivity.csv")))
    print(head(da.peaks))
    write.csv(da.peaks, file=file.path(clust_outdir, paste0("cluster_",i,".DE.GeneActivity.csv")))
    print(paste('saved cluster', i))
    plotDE(integrated, da.peaks, i, clust_outdir)
    print(paste('plotted cluster', i))
    
})
}

[1] "cluster 0"
[1] "/data2/mito_lineage/output/annotation/data/jan21_2021/MTblacklist/mergedSamples/DE_minPct01/clusters/cluster_0.DE.GeneActivity.csv"
              p_val avg_log2FC pct.1 pct.2     p_val_adj
CPM   6.098656e-299  0.6614416 0.747 0.507 1.220341e-294
EDIL3 7.857709e-239  0.6204164 0.786 0.601 1.572328e-234
RBM47 2.227840e-114  0.3881796 0.648 0.481 4.457908e-110
TPRG1 2.838994e-111  0.3981779 0.764 0.626 5.680826e-107
DLG4  8.180332e-110  0.2567312 0.288 0.127 1.636884e-105
NUP37 3.500970e-106  0.3013632 0.342 0.165 7.005441e-102
[1] "saved cluster 0"


Saving 6.67 x 6.67 in image



[1] "plotted cluster 0"
[1] "cluster 1"
[1] "/data2/mito_lineage/output/annotation/data/jan21_2021/MTblacklist/mergedSamples/DE_minPct01/clusters/cluster_1.DE.GeneActivity.csv"
               p_val avg_log2FC pct.1 pct.2     p_val_adj
USP32  1.936685e-245  0.5857602 0.936 0.709 3.875306e-241
PRTN3  3.273367e-166  0.2696656 0.300 0.068 6.550008e-162
AMOTL1  8.070578e-94  0.3362583 0.791 0.541  1.614923e-89
RTBDN   2.920090e-93  0.2994694 0.520 0.270  5.843100e-89
FNDC3B  2.558275e-91  0.3181998 0.928 0.758  5.119109e-87
LYST    1.322279e-78  0.3006559 0.850 0.644  2.645881e-74
[1] "saved cluster 1"


Saving 6.67 x 6.67 in image



[1] "plotted cluster 1"
[1] "cluster 2"
[1] "/data2/mito_lineage/output/annotation/data/jan21_2021/MTblacklist/mergedSamples/DE_minPct01/clusters/cluster_2.DE.GeneActivity.csv"
                p_val avg_log2FC pct.1 pct.2     p_val_adj
MAP4K4  2.799787e-158  0.5392473 0.623 0.405 5.602373e-154
NUCKS1  1.396591e-113  0.3460355 0.415 0.211 2.794578e-109
RAB7A   2.585884e-111  0.3881060 0.507 0.301 5.174353e-107
HHAT     1.954714e-98  0.4020087 0.663 0.491  3.911383e-94
SLC45A3  1.363642e-94  0.2697967 0.273 0.112  2.728647e-90
BIRC6    2.705290e-94  0.4142714 0.673 0.506  5.413285e-90
[1] "saved cluster 2"


...

[1] "plotted cluster 12"
[1] "cluster 13"
[1] "/data2/mito_lineage/output/annotation/data/jan21_2021/MTblacklist/mergedSamples/DE_minPct01/clusters/cluster_13.DE.GeneActivity.csv"
                  p_val avg_log2FC pct.1 pct.2    p_val_adj
PACS2      1.245541e-28  0.4820892 0.422 0.177 2.492327e-24
LDLRAD4    7.706465e-21  0.5163279 0.798 0.689 1.542064e-16
CSGALNACT1 1.375822e-18  0.5199520 0.650 0.524 2.753021e-14
STK17A     2.368764e-17  0.3927834 0.350 0.200 4.739897e-13
CCDC138    3.405421e-17  0.3589479 0.329 0.181 6.814247e-13
UBL3       4.517087e-17  0.4714435 0.458 0.324 9.038690e-13
[1] "saved cluster 13"


Saving 6.67 x 6.67 in image



[1] "plotted cluster 13"
[1] "cluster 14"
[1] "/data2/mito_lineage/output/annotation/data/jan21_2021/MTblacklist/mergedSamples/DE_minPct01/clusters/cluster_14.DE.GeneActivity.csv"
               p_val avg_log2FC pct.1 pct.2    p_val_adj
ZNF423  2.220235e-33  0.5190860 0.912 0.450 4.442691e-29
TTC34   9.560897e-33  0.4393915 0.747 0.220 1.913136e-28
RAPGEF1 6.453819e-32  0.4568475 0.760 0.233 1.291409e-27
TTLL8   6.886477e-31  0.3701378 0.604 0.114 1.377984e-26
SLC4A9  1.160903e-30  0.3178940 0.502 0.068 2.322968e-26
FNDC3B  3.842937e-30 -0.7711152 0.788 0.777 7.689716e-26
[1] "saved cluster 14"


Saving 6.67 x 6.67 in image



[1] "plotted cluster 14"


In [ ]:
library(stats)
p.adjust(p, method = p.adjust.methods, n = length(p))
p.adjust.methods
# c("holm", "hochberg", "hommel", "bonferroni", "BH", "BY",
#   "fdr", "none")

## Loop through each cluster and run DA for ATAC-seq

In [ ]:
DefaultAssay(integrated) <- "ATAC"
cluster.ids <- sort(unique(integrated$seurat_clusters))
for (i in cluster.ids){ #or however many clusters you have
try({
    print(paste("cluster", i))
    da.peaks <- FindMarkers(
                  object = integrated,
                  ident.1 = i, #"CD4 Naive",
                  min.pct = minPct,
                  test.use = 'LR', latent.vars = 'peak_region_fragments'
                )
    print(head(da.peaks))
    write.csv(da.peaks, file=file.path(clust_atac_outdir, paste0("cluster_",i,".DE.PeakActivity.csv")))
    print(paste('saved cluster', i))
    plotDE(integrated, da.peaks, i, clust_atac_outdir)
    print(paste('plotted cluster', i))
    
})
}

# put back to RNA
DefaultAssay(integrated) <- "RNA"


[1] "cluster 0"
                                 p_val avg_log2FC pct.1 pct.2     p_val_adj
chr12-69351650-69358100   0.000000e+00  0.9736617 0.426 0.076  0.000000e+00
chr12-14613607-14615284  1.660341e-105  0.5050353 0.134 0.028 2.482791e-100
chr5-80125088-80127605   4.441525e-103  0.5020980 0.216 0.074  6.641635e-98
chr18-76499399-76500579   9.001010e-92  0.4744257 0.136 0.033  1.345966e-86
chr5-99314778-99316249    1.698990e-89  0.4452327 0.095 0.016  2.540584e-84
chr4-121168552-121170293  1.779034e-86  0.4554631 0.121 0.029  2.660278e-81
[1] "saved cluster 0"


Saving 6.67 x 6.67 in image



[1] "plotted cluster 0"
[1] "cluster 1"
                                 p_val avg_log2FC pct.1 pct.2     p_val_adj
chr19-845627-852730      1.006402e-230  0.8323580 0.379 0.082 1.504923e-225
chr19-854064-861070      4.274078e-163  0.7223659 0.317 0.078 6.391242e-158
chr10-69072718-69076229  1.799802e-105  0.5995454 0.256 0.072 2.691334e-100
chr2-227817934-227819348 1.359338e-100  0.5795030 0.288 0.093  2.032687e-95
chr19-829306-833271       7.370994e-98  0.5738283 0.154 0.027  1.102222e-92
chr2-31032332-31033442    1.057236e-68  0.4869293 0.135 0.029  1.580937e-63
[1] "saved cluster 1"


Saving 6.67 x 6.67 in image



[1] "plotted cluster 1"
[1] "cluster 2"
                                 p_val avg_log2FC pct.1 pct.2     p_val_adj
chr19-39729885-39731979  5.505998e-222  0.7790801 0.145 0.005 8.233394e-217
chr9-89501787-89502632   5.222332e-159  0.6626586 0.130 0.009 7.809214e-154
chr2-28557318-28558306   8.425246e-136  0.6037149 0.110 0.007 1.259869e-130
chr21-41148854-41149641  6.035502e-130  0.5714177 0.093 0.004 9.025188e-125
chr5-121182638-121183498 2.772542e-115  0.5728733 0.108 0.010 4.145921e-110
chr5-134714246-134715274 7.626711e-113  0.5407907 0.092 0.006 1.140460e-107
[1] "saved cluster 2"


...

Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"

                                 p_val avg_log2FC pct.1 pct.2 p_val_adj
chr11-108220541-108225642 2.618016e-07 -0.4900496 0.035 0.200 0.0391485
chr6-37168971-37178447    2.137586e-06 -0.5180831 0.077 0.307 0.3196439
chr11-59614563-59616991   3.389727e-06 -0.4950477 0.058 0.244 0.5068829
chr6-33787786-33789517    7.196433e-06 -0.2727316 0.000 0.050 1.0000000
chr15-50259234-50263338   1.060988e-05 -0.3642287 0.016 0.102 1.0000000
chr6-152982307-152985321  1.066466e-05 -0.4354006 0.029 0.161 1.0000000
[1] "saved cluster 11"


Saving 6.67 x 6.67 in image



[1] "plotted cluster 11"
[1] "cluster 12"
                                p_val avg_log2FC pct.1 pct.2    p_val_adj
chr15-38129428-38131579  6.982245e-49  0.9280846 0.336 0.046 1.044090e-43
chr9-127838347-127839158 1.774466e-45  0.8729566 0.181 0.010 2.653447e-40
chr10-89235521-89235817  3.320843e-42  0.7553711 0.119 0.003 4.965823e-37
chr3-10173347-10174501   4.587815e-37  0.8255455 0.220 0.023 6.860389e-32
chr14-75609947-75610428  6.204799e-37  0.7351415 0.129 0.007 9.278347e-32
chr1-109821055-109822846 7.181187e-37  0.8350549 0.245 0.030 1.073839e-31
[1] "saved cluster 12"


Saving 6.67 x 6.67 in image



## Stimulus specific response

In [ ]:
stimout <- function(outdir, sample_names){
    cluster.ids <- sort(unique(integrated$seurat_clusters))
    integrated$celltype.stim <- paste(integrated$seurat_clusters, integrated$orig.ident, sep = "_")
    integrated$celltype <- integrated$seurat_clusters
    Idents(integrated) <- "celltype.stim"

    for (c in cluster.ids){
        try({
            response <- FindMarkers(integrated, 
                                    ident.1 = paste0(c, "_", sample_names[[1]]), 
                                    ident.2 = paste0(c, "_", sample_names[[2]]), 
                                    verbose = FALSE,
                                    test.use = 'LR', min.pct = minPct,
                                    latent.vars = 'peak_region_fragments'
                                   )
            if (!(dim(response)[1]==0)){

                print(head(response, n = 15))
                curr_clust <- subset(integrated, seurat_clusters == c)
                avg_curr_clust <- data.frame(log1p(AverageExpression(curr_clust, verbose = FALSE)$RNA))
                avg_curr_clust$gene <- rownames(avg_curr_clust)

                p1 <- ggplot(avg_curr_clust, aes_string(paste0("X", c, "_", sample_names[[1]]), paste0("X", c, "_", sample_names[[2]]))) + geom_point() + ggtitle(paste("Cluster", c))
                p1 <- LabelPoints(plot = p1, points = rownames(head(response, n = 15)), repel = TRUE)
                plot_grid(p1)
                write.csv(response, file=file.path(outdir,paste0("cluster_",i,".conditionDE.csv")))
                ggsave(file.path(outdir,paste0("cluster_",i,".conditionScatter.png")))

            }
        })

    }
}



In [ ]:
if ((comps_f=="") | comps_f == "NULL"){
    stimout(cond_outdir, sample_names)

}else{
    comps <- unlist(strsplit(comps_f, ";")[[1]])
    comps <- lapply(comps, function(x) unlist(strsplit(x, ',')[[1]]))
    for (i in comps){
        print(i)
        curr_sample_names <- i[2:3]
        curr_cond_outdir <- file.path(cond_outdir, i[1])
        dir.create(curr_cond_outdir, showWarnings = FALSE)
        stimout(curr_cond_outdir, curr_sample_names)
    }
}

# comps_f <- "A,preA,postA"#;B,preB,postB"
# comps <- unlist(strsplit(comps_f, ";")[[1]])
# #samples <- unlist(strsplit(samples, ","))

# comps <- lapply(comps, function(x) unlist(strsplit(x, ',')[[1]]))
# comps
# for (i in comps){
#     print(i)
#     sample_names <- i[2:3]
# }
# sample_names


                    
# cluster.ids <- sort(unique(integrated$seurat_clusters))
# integrated$celltype.stim <- paste(integrated$seurat_clusters, integrated$orig.ident, sep = "_")
# integrated$celltype <- integrated$seurat_clusters
# Idents(integrated) <- "celltype.stim"

# for (c in cluster.ids){
#     try({
#         response <- FindMarkers(integrated, 
#                                 ident.1 = paste0(c, "_", sample_names[[1]]), 
#                                 ident.2 = paste0(c, "_", sample_names[[2]]), 
#                                 verbose = FALSE,
#                                 test.use = 'LR', min.pct = 0.1,
#                                 latent.vars = 'peak_region_fragments'
#                                )
#         if (!(dim(response)[1]==0)){

#             print(head(response, n = 15))
#             curr_clust <- subset(integrated, seurat_clusters == c)
#             avg_curr_clust <- data.frame(log1p(AverageExpression(curr_clust, verbose = FALSE)$RNA))
#             avg_curr_clust$gene <- rownames(avg_curr_clust)

#             p1 <- ggplot(avg_curr_clust, aes_string(paste0("X", c, "_", sample_names[[1]]), paste0("X", c, "_", sample_names[[2]]))) + geom_point() + ggtitle(paste("Cluster", c))
#             p1 <- LabelPoints(plot = p1, points = rownames(head(response, n = 15)), repel = TRUE)
#             plot_grid(p1)
#             write.csv(response, file=file.path(cond_outdir,paste0("cluster_",i,".conditionDE.csv")))
#             ggsave(file.path(cond_outdir,paste0("cluster_",i,".conditionScatter.png")))

#         }
#     })

# }

## Gene Markers plot

In [ ]:
clrs =  brewer.pal(n = length(sample_names), name = "RdBu")

### Immune markers taken from Seurat

In [ ]:
Idents(integrated) <- "seurat_clusters"

In [ ]:
FeaturePlot(integrated, features = c("CD3D", "SELL", "CREM", "CD8A", "GNLY", "CD79A", "FCGR3A", 
    "CCL2", "PPBP"), min.cutoff = "q9")

ggsave(file=file.path(outdir,"seuratImmuneEmbed.png"))


In [ ]:
markers.to.plot <- c("CD3D", "CREM", "HSPH1", "SELL", "GIMAP5", "CACYBP", "GNLY", "NKG7", "CCL5", 
    "CD8A", "MS4A1", "CD79A", "MIR155HG", "NME1", "FCGR3A", "VMO1", "CCL2", "S100A9", "HLA-DQA1", 
    "GPR183", "PPBP", "GNG11", "HBA2", "HBB", "TSPAN13", "IL3RA", "IGJ")
DotPlot(integrated, features = rev(markers.to.plot), dot.scale = 8, cols=clrs,  
    split.by = "orig.ident") + RotatedAxis()
ggsave(file=file.path(outdir,"seuratImmuneDotPlot.png"))

### Immune markers taken from Dawn Lin et al Flt3l paper

In [ ]:
immune.markers = c('KIT',
                   'LY6E',
                   'ITGAX',
                   'SLAMF1',
                   'CD34',
                   'FCGR3A',
                   'PTPRC',
                   'SLAMF2',
                   'IL7R',
                   'ITGAM')

DotPlot(integrated, features = rev(immune.markers), dot.scale = 8, 
        cols = clrs,
    split.by = "orig.ident") + RotatedAxis()
ggsave(file=file.path(outdir,"linImmuneDotPlot.png"))

In [ ]:
FeaturePlot(integrated, features = immune.markers, min.cutoff = "q9")
ggsave(file=file.path(outdir,"linImmuneEmbed.png"))

## Conserved markers to help identify cell type

In [ ]:
topMarkers = c()
for (i in cluster.ids){ #or however many clusters you have
try({
    print(paste("cluster", i))
    cons.markers <- FindConservedMarkers(integrated, ident.1 = i, 
                                         grouping.var = "orig.ident", verbose = TRUE)

    write.csv(cons.markers, file=file.path(cons_outdir,paste0("cluster_",i,".conservedOverStim.GeneActivity.csv")))
    print(paste('saved cluster', i))
    print(head(cons.markers))
    topMarkers = c(topMarkers, rownames(cons.markers)[1])
    print(topMarkers)
})
}

head(cons.markers)

In [ ]:
sessionInfo()